# VAE version 001

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
data_input_size = (46, 22499)

Define network

In [5]:
class VGGStyleNetwork1D(nn.Module):
    def __init__(self, embedding_size):
        super(VGGStyleNetwork1D, self).__init__()
        self.features = nn.Sequential(
            # conv1
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv2
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv3
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2),
            # conv4
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        self.flatten = nn.Flatten()
        
        # Calculate the flattened size after the convolutional layers
        # For input (1, 46, 22499), the size is derived based on the stride and pooling layers.
        # Adjust this size as needed after inspecting the output dimensions.
        self.classifier = nn.Sequential(
            # fc1
            nn.Linear(512 * 2 * 1406, 4096),  # Update based on feature map dimensions
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            # fc2
            nn.Linear(4096, embedding_size)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

# Example usage
embedding_size = 128  # Define the embedding size
model = VGGStyleNetwork1D(embedding_size)

# Example input: single channel, 46x22499
input_data = torch.randn(1, 46, 22499)
output = model(input_data.squeeze(1))  # Adjust for 1D input
print(output.shape)


: 